### Vamos a crear un script que suba de forma automática los datos

In [18]:
import pandas as pd  # transformar y leer excel como tabla (dataframe)
import re # regular expressions, expresiones regulares para trabajar texto
import mysql.connector as maria # conectarnos a la base de datos MariaDB

In [19]:
dataframe = pd.read_excel('datos/datos_proveedores.xlsx',
                          sheet_name='Alimentacion',skiprows=1)

In [20]:
dataframe.head(1)

,N°,Id Articulo,Descripción,SubCategoria,Nombre Categoria,Id Proveedor,Nombre Proveedor,Moneda,Precio,Monto/Cantidad Minima,Tiempo de Entrega,Fecha de vencimiento,ZONA
0,1,UDLA_ALMUERZO_0001,"Almuerzo 1 pechuga pavo o pollo,mix ensaladas,...",S030805,Servicios de comida y bebida para empleados,76961994-1,CAMASI SPA,CLP,4500,35000,2,2023-04-05,Viña


dataframe.describe()

dataframe.dtypes

dataframe.columns

dataframe.columns[:13]

In [3]:
conexion = maria.connect(user='root', 
                         password='aci253udla', 
                         host='localhost', 
                         database='proveedores', 
                         port='3306')

In [21]:
dataframe['ZONA']

0               Viña
1               Viña
2               Viña
3               Viña
4               Viña
           ...      
149    Viña del Mar 
150    Viña del Mar 
151    Viña del Mar 
152    Viña del Mar 
153    Viña del Mar 
Name: ZONA, Length: 154, dtype: object

In [22]:
dataframe['ZONA'].unique()

array(['Viña', 'Concepcion ', 'Santiago ', 'Santiago', 'Viña del Mar '],
      dtype=object)

id_zona=0;
for ciudad in dataframe['ZONA'].unique():
    id_zona+=1
    print(id_zona,ciudad)

for ciudad in dataframe['ZONA']:
    if ciudad == 'Viña':
        ciudad = 'Viña del Mar'
    st=re.sub(r"\s+$", "", ciudad);
    print(ciudad,st)

In [24]:
dataframe['ZONA'].replace(to_replace='Viña', 
                          value='Viña del Mar',inplace=True)

In [25]:
dataframe['ZONA'].replace(to_replace='Santiago ',
                          value='Santiago', inplace=True)

In [26]:
dataframe['ZONA'].replace(to_replace='Viña del Mar ',
                          value='Viña del Mar', inplace=True)

In [27]:
dataframe['ZONA'].replace(to_replace='Concepcion ',
                          value='Concepcion', inplace=True)

In [28]:
agregar_zona =("INSERT INTO zona ""(id,nombre)"" VALUES (%s,%s)")

In [29]:
print(agregar_zona)

INSERT INTO zona (id,nombre) VALUES (%s,%s)


#### Creamos buffer para la conexión con la base de datos

In [11]:
buffer = conexion.cursor(buffered=True)

In [30]:
idzona=1
for zona in dataframe['ZONA'].unique():
    value_insert=(idzona,zona)
    print(value_insert)
    buffer.execute(agregar_zona,value_insert)
    idzona+=1;


(1, 'Viña del Mar')
(2, 'Concepcion')
(3, 'Santiago')


In [ ]:
conexion.commit()

In [ ]:
columnas_proveedor=['Id Proveedor','Nombre Proveedor']

In [ ]:
dfproveedor=dataframe[columnas_proveedor]

In [ ]:
dfproveedor.drop_duplicates(inplace=True)

In [ ]:
dfproveedor

In [ ]:
dfproveedor

In [ ]:
agregar_proveedores =("INSERT INTO proveedor ""(id,rut,nombre)"" VALUES (%s,%s,%s)")

In [ ]:
idproveedor=1
for proveedor in dfproveedor.values:
    prov_insert=(idproveedor,proveedor[0],proveedor[1])
    print(prov_insert)
    buffer.execute(agregar_proveedores,prov_insert)
    idproveedor+=1


In [ ]:
dataframe.columns

In [ ]:
cols=['Id Proveedor','Fecha de vencimiento','ZONA']

In [ ]:
dataframe[cols].drop_duplicates()

In [ ]:
seleccionar_proveedorID =("SELECT id FROM proveedor WHERE rut=""(%s)""")
seleccionar_zonaID=("SELECT id FROM zona WHERE nombre=""(%s)""")

In [ ]:
agregar_zona_proveedores =("INSERT INTO zona_proveedor ""(id,proveedorID,zonaID,vencimiento)"" VALUES (%s,%s,%s,%s)")

In [ ]:
zona_proveedorID=1;
for proveedor in dataframe[cols].drop_duplicates().values:
    prov_rut=(proveedor[0])
    zona_nombre=(proveedor[2])
    #print(type(prov_rut))
    #print(prov_rut)
    buffer.execute(seleccionar_proveedorID,[prov_rut])
    pID=buffer.fetchone()
    buffer.execute(seleccionar_zonaID,[zona_nombre])
    zID=buffer.fetchone()
    valores_zp=(zona_proveedorID,pID[0],zID[0],proveedor[1])
    buffer.execute(agregar_zona_proveedores,valores_zp)
    zona_proveedorID+=1;

In [ ]:
conexion.commit()

In [ ]:
agregar_categoria =("INSERT INTO categoria ""(id,proveedorID,nombre)"" VALUES (%s,%s,%s)")

In [ ]:
categoriaID=1;
for categoria in dataframe[['Nombre Categoria','Id Proveedor']].drop_duplicates().values:
    prov_rut=(categoria[1])
    categoria_nombre=(categoria[0])
    #print(type(prov_rut))
    #print(prov_rut)
    buffer.execute(seleccionar_proveedorID,[prov_rut])
    pID=buffer.fetchone()
    valores_cat=(categoriaID,pID[0],categoria_nombre)
    buffer.execute(agregar_categoria,valores_cat)
    categoriaID+=1;

In [ ]:

buffer.close()
conexion.close()

In [ ]:
dataframe.columns